### Step 4.1: Create score.py

In [1]:
%%writefile score.py
import json
import pickle
import numpy as np
from azureml.core.model import Model

def init():
    global model
    global dv

    model_path = Model.get_model_path("nyc-taxi-best-model")
    with open(model_path, "rb") as f:
        model = pickle.load(f)

    dv_path = Model.get_model_path("nyc-taxi-preprocessor")
    with open(dv_path, "rb") as f:
        dv = pickle.load(f)

def run(raw_data):
    try:
        data = json.loads(raw_data)["data"]
        X = dv.transform(data)
        preds = model.predict(X)
        return preds.tolist()
    except Exception as e:
        return str(e)


Writing score.py


### Step 4.2: Create environment.yml

In [2]:
%%writefile environment.yml
name: nyc-taxi-env
dependencies:
  - python=3.8
  - scikit-learn
  - pandas
  - pip
  - pip:
      - azureml-defaults


Writing environment.yml


### Step 4.3: Deploy the model

In [6]:
from azureml.core import Workspace, Environment, Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import InferenceConfig

ws = Workspace.from_config()

# Use previously registered model names
model = Model(ws, name="nyc-taxi-best-model")
dv_model = Model(ws, name="nyc-taxi-preprocessor")

# Define environment
env = Environment.from_conda_specification(name="nyc-taxi-env", file_path="environment.yml")

# Define scoring config
inference_config = InferenceConfig(entry_script="score.py", environment=env)

# Define deployment config (ACI = simple test deployment)
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Deploy service
service = Model.deploy(
    workspace=ws,
    name="nyc-taxi-endpoint",
    models=[model, dv_model],
    inference_config=inference_config,
    deployment_config=deployment_config,
    overwrite=True
)




/tmp/ipykernel_5031/2096244599.py:21: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


In [7]:
service.wait_for_deployment(show_output=True)
print(f"✅ Deployment state: {service.state}")
print(f"🔗 Scoring URI: {service.scoring_uri}")

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-05-09 11:32:10+00:00 Creating Container Registry if not exists.
2025-05-09 11:32:11+00:00 Use the existing image.
2025-05-09 11:32:13+00:00 Submitting deployment to compute.
Failed


Service deployment polling reached non-successful terminal state, current service state: Transitioning
Operation ID: 54f4314a-689f-42a4-9afa-733e04bcce8a
More information can be found using '.get_logs()'
Error:
{
  "code": "AuthorizationFailed",
  "statusCode": 403,
  "message": "ACI Service request failed. Reason: The client 'f7fb941e-bf50-443e-9316-6bcef3a1e5ee' with object id 'f7fb941e-bf50-443e-9316-6bcef3a1e5ee' does not have authorization to perform action 'Microsoft.ContainerInstance/containerGroups/write' over scope '/subscriptions/6acb6bbc-97bd-42b6-b199-75e4d94b2600/resourceGroups/mlops/providers/Microsoft.ContainerInstance/containerGroups/nyc-taxi-endpoint-q7Khr-U00UqvP9cbOgM1gw' or the scope is invalid. If access was recently granted, please refresh your credentials.."
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Transitioning
Operation ID: 54f4314a-689f-42a4-9afa-733e04bcce8a
More information can be found using '.get_logs()'
Error:
{
  "code": "AuthorizationFailed",
  "statusCode": 403,
  "message": "ACI Service request failed. Reason: The client 'f7fb941e-bf50-443e-9316-6bcef3a1e5ee' with object id 'f7fb941e-bf50-443e-9316-6bcef3a1e5ee' does not have authorization to perform action 'Microsoft.ContainerInstance/containerGroups/write' over scope '/subscriptions/6acb6bbc-97bd-42b6-b199-75e4d94b2600/resourceGroups/mlops/providers/Microsoft.ContainerInstance/containerGroups/nyc-taxi-endpoint-q7Khr-U00UqvP9cbOgM1gw' or the scope is invalid. If access was recently granted, please refresh your credentials.."
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current servic

### Step 4.4: Test the endpoint

In [8]:
import requests
import json

sample_data = {
    "data": [
        {"PU_DO": "129_205", "trip_distance": 3.5}
    ]
}

headers = {"Content-Type": "application/json"}

response = requests.post(service.scoring_uri, headers=headers, data=json.dumps(sample_data))
print("✅ Prediction result:", response.json())


MissingSchema: Invalid URL 'None': No scheme supplied. Perhaps you meant https://None?